# Training a LoRA Adapter for Ubersicht React/JSX Widgets on Colab

Trains a LoRA adapter for Apple Foundation Models (v26.0.0) to specialize a chatbot in React/JSX widget generation using 434 hand-crafted widgets. Runs on Colab’s Linux GPU (T4/A100) with MLX CUDA backend. Matches Apple’s `end_to_end_example.ipynb` workflow.

## Prerequisites
- Colab GPU runtime (free T4 or Pro A100).
- Apple Developer Program toolkit in `/content/drive/MyDrive/toolkit/`.
- 434 widget files in `/content/drive/MyDrive/widgets/`.
- Hugging Face token.
- Python 3.11+.

## Step 1: Setup

Mount Drive, install dependencies, verify GPU, set memory optimization.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
sys.path.append(os.path.abspath('/content/drive/MyDrive/AITraining'))
sys.path.append(os.path.abspath('/content/drive/MyDrive/AITraining/examples'))

Mounted at /content/drive


In [ ]:
# Cell 1
!ls -la /content/drive/MyDrive/AITraining/examples/__init__.py

ls: cannot access '/content/AITraining/examples/__init__.py': No such file or directory


In [ ]:
# Install from GPU requirements file first (gets all dependencies)
!pip install -r /content/drive/MyDrive/AITraining/requirements_GPU_modified.txt

print("Checking tamm (Apple Foundation Models) PyTorch dependency...")
print("="*60)
!pip show tamm 2>&1 | grep -A 15 "Requires:" || echo "Could not determine tamm requirements"
print("="*60)

# Check installed PyTorch version (should be >=2.6 from requirements_GPU.txt)
print("\n" + "="*60)
print("Checking installed PyTorch version...")
print("="*60)
!pip show torch 2>&1 | grep -E "(Name|Version)" || echo "Could not determine torch version"
print("="*60)


# Optimize memory
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'  # Disable MPS limit
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"      # optional, speeds up downloads
os.environ["HF_LIST_MODELS_ALL"] = "1"             # <-- this is the crucial line


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [ ]:
# Check installed versions
import subprocess

print("="*60)
print("Checking installed versions...")
print("="*60)

# Check cuDNN version
result = subprocess.run(['pip', 'show', 'nvidia-cudnn-cu12'], capture_output=True, text=True)
if result.returncode == 0:
    for line in result.stdout.split('\n'):
        if 'Version:' in line:
            print(f"📦 cuDNN version: {line.split(':')[1].strip()}")
            break

# Check MLX version
result = subprocess.run(['pip', 'show', 'mlx'], capture_output=True, text=True)
if result.returncode == 0:
    for line in result.stdout.split('\n'):
        if 'Version:' in line:
            print(f"📦 MLX version: {line.split(':')[1].strip()}")
            break

# Check CUDA packages
result = subprocess.run(['pip', 'show', 'nvidia-cublas-cu12'], capture_output=True, text=True)
if result.returncode == 0:
    for line in result.stdout.split('\n'):
        if 'Version:' in line:
            print(f"📦 CUDA cublas version: {line.split(':')[1].strip()}")
            break

print("="*60)

# MLX check - MLX is crucial for Apple Foundation Models
# If this fails or uses CPU, training may not work properly
try:
    import mlx.core as mx
    device = mx.default_device()
    print(f"\n✅ MLX import successful")
    print(f"📱 MLX device: {device}")
    
    # Check if it's actually GPU
    if hasattr(device, 'gpu') or str(device).startswith('gpu') or 'gpu' in str(device).lower():
        print("✅ MLX is using GPU - ready for Apple Foundation Models")
    else:
        print(f"❌ CRITICAL: MLX is using {device}, not GPU!")
        print("   This will cause training to fail or be extremely slow.")
        print("   Possible causes:")
        print("   - MLX version too old (need 0.30.3)")
        print("   - cuDNN/CUDA version mismatch")
        print("   - GPU not available or not detected")
        print("\n   Try restarting runtime and re-running installation cells")
        raise RuntimeError(f"MLX is not using GPU: {device}")
        
except ImportError as e:
    print(f"❌ CRITICAL: MLX import failed: {e}")
    print("   This may prevent Apple Foundation Models from working properly.")
    print("   Possible causes:")
    print("   - cuDNN version mismatch (MLX 0.30.3 needs cuDNN 9.18+ with cudnnBackendPopulateCudaGraph)")
    print("   - CUDA version mismatch")
    print("   Solutions:")
    print("   1. Restart runtime and re-run installation cells")
    print("   2. Check: !pip show mlx mlx-cuda-12")
    raise  # Re-raise to make the error obvious

ImportError: /usr/local/lib/python3.12/dist-packages/mlx/lib/libmlx.so: undefined symbol: cudnnBackendPopulateCudaGraph, version libcudnn.so.9

In [6]:
!huggingface-cli login  # Enter token
!nvidia-smi  # Verify GPU


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Google Colaboratory` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /r

In [7]:
import sys
import os
import importlib.util

# Remove the problematic path additions
if '/content/drive/MyDrive/AITraining/examples' in sys.path:
    sys.path.remove('/content/drive/MyDrive/AITraining/examples')

# Add the parent directory to sys.path so Python can find the examples package
aitraining_path = '/content/drive/MyDrive/AITraining'
if aitraining_path not in sys.path:
    sys.path.insert(0, aitraining_path)

print("🔍 Updated sys.path:")
for i, path in enumerate(sys.path[:5]):  # Show first 5 paths
    print(f"  {i}: {path}")

# Now try importing the package properly
try:
    # Import the examples package
    import examples

    # Force reload to ensure fresh import
    importlib.reload(examples)

    print("✅ Examples package imported successfully")

    # Now try to import the submodules
    from examples import generate
    from examples import messages

    print("✅ Submodules imported successfully")

    # Finally, try the specific functions
    from examples.generate import generate_content, GenerationConfiguration
    from examples.messages import Message

    print("✅ Function imports successful!")
    print("🎉 All imports working!")

except ImportError as e:
    print(f"❌ Import failed: {e}")

    # Let's try a different approach - check what's in the __init__.py
    init_file = '/content/drive/MyDrive/AITraining/examples/__init__.py'
    with open(init_file, 'r') as f:
        init_content = f.read()
        print(f"\n🔍 __init__.py content:")
        print(init_content[:500] + "..." if len(init_content) > 500 else init_content)

🔍 Updated sys.path:
  0: /content
  1: /env/python
  2: /usr/lib/python312.zip
  3: /usr/lib/python3.12
  4: /usr/lib/python3.12/lib-dynload
✅ Examples package imported successfully
✅ Submodules imported successfully
✅ Function imports successful!
🎉 All imports working!


In [8]:
import torch
print(f"GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"GPU type: {torch.cuda.get_device_name(0)}")

GPU memory: 0.00 GB
GPU type: NVIDIA A100-SXM4-40GB


## Step 2: Test Generation

Verify setup with base model.

In [ ]:
# Clear GPU memory before training
import torch
import gc

# Clear PyTorch cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# Check available memory
if torch.cuda.is_available():
    print(f"GPU Memory - Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU Memory - Reserved: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    print(f"GPU Memory - Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1024**3:.2f} GB")

In [9]:
# These imports will be available in Colab after toolkit upload
try:
    from examples.generate import generate_content, GenerationConfiguration
    from examples.messages import Message
except ImportError:
    print("⚠️  Toolkit not yet uploaded to Google Drive")
    print("   These imports will work in Colab after upload")


# Force model to stay in memory
import torch
torch.cuda.empty_cache()  # Clear any fragmented memory first

# Then run generation - model should stay loaded
output = generate_content(
    [[Message.from_system('You are a helpful assistant generating React/JSX widgets.'),
      Message.from_user('Create a React button widget')]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=128)
)

print(f"After generation: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(output[0].response)

After generation: 0.01 GB
Certainly! Below is a simple example of a React button widget. This button will display a message when clicked and change its color based on the state of a boolean variable.

```jsx
import React, { useState } from 'react';

const ButtonWidget = () => {
  const [isActive, setIsActive] = useState(false);

  const handleClick = () => {
    setIsActive(!isActive);
  };

  return (
    <div>
      <button
        style={{
          backgroundColor: isActive ? '


## Step 3: Train Adapter



Select dataset


In [10]:
#DATA_SET = 'test_complete'
#DATA_SET = 'withtoolcall'
#DATA_SET = 'toolcall_plus_notruncation'
#DATA_SET = 'with_tool_calls_4'
#DATA_SET = 'fixed_create_directive_bias'
#DATA_SET = 'API_props_are_optional'
DATA_SET = 'minimal_system_prompt'

Run the training





In [ ]:
TRAIN_FILE = f'/content/drive/MyDrive/AITraining/US_E2E/training_sets/{DATA_SET}/train.jsonl'
VALID_FILE = f'/content/drive/MyDrive/AITraining/US_E2E/training_sets/{DATA_SET}/valid.jsonl'
# TEST_FILE = f'/content/drive/MyDrive/AITraining/US_E2E/training_sets/{DATA_SET}/test.jsonl'

from examples.train_adapter import AdapterTrainingConfiguration, train_adapter

config1=AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=1,  # Reduce from 4 to 1
    gradient_accumulation_steps=8,  # Compensate by accumulating gradients
    enable_activation_checkpointing=True,
    max_sequence_length=256,  # Reduce further
    compile_model=False  # Disable compilation
)

config2 = AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=2,  # Try increasing if memory allows
    gradient_accumulation_steps=4,  # Reduce accordingly
    enable_activation_checkpointing=True,
    precision='bf16-mixed',
    max_sequence_length=512,  # Increase for better context
    compile_model=False
)

config2 = AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=2,  # Try increasing if memory allows
    gradient_accumulation_steps=4,  # Reduce accordingly
    enable_activation_checkpointing=True,
    precision='bf16-mixed',
    max_sequence_length=512,  # Increase for better context
    compile_model=False
)

config3 = AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=2,  # Try increasing if memory allows
    gradient_accumulation_steps=4,  # Reduce accordingly
    enable_activation_checkpointing=True,
    precision='bf16-mixed',
    max_sequence_length=4095,
    compile_model=False
)

# Memory-efficient config for A100 (40GB) - fixes OOM error
# Reduced max_sequence_length from 4095 to 2048, batch_size to 1
config4 = AdapterTrainingConfiguration(
    epochs=6,
    learning_rate=1e-4,
    batch_size=1,  # Reduced from 2 to save memory
    gradient_accumulation_steps=8,  # Increased from 4 to maintain effective batch size (1*8=8)
    enable_activation_checkpointing=True,
    precision='bf16-mixed',
    max_sequence_length=2048,  # Reduced from 4095 - still provides good context
    compile_model=False,
    fixed_sized_sequences=True  # More memory efficient than variable length
)

train_adapter(
    train_data=TRAIN_FILE,
    eval_data=VALID_FILE,
    config=config4,  # Using config4 for memory-efficient training
    checkpoint_dir='/content/drive/MyDrive/checkpoints'
)

INFO:examples.utils:Fine-tuning adapters with configuration: 
AdapterTrainingConfiguration(epochs=6, learning_rate=0.0001, batch_size=2, linear_warmup_epochs=1, gradient_accumulation_steps=4, enable_activation_checkpointing=True, precision='bf16-mixed', compile_model=False, weight_decay=0.01, clip_grad_norm=1.0, max_sequence_length=4095, fixed_sized_sequences=False, pack_sequences=False, loss_update_frequency=3)
INFO:examples.utils:Loading base model on cuda with precision torch.float32
INFO:examples.utils:Total parameters 3178001792
INFO:examples.utils:Total trainable parameters 66633728
INFO:examples.utils:Gradient scaling is enabled: False
INFO:examples.utils:Epoch 1/6
Training:   0%|          | 0/25 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.46 GiB. GPU 0 has a total capacity of 39.56 GiB of which 2.47 GiB is free. Process 42286 has 37.08 GiB memory in use. Of the allocated memory 35.92 GiB is allocated by PyTorch, and 672.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Step 5: Train Draft Model (Optional)

Train draft model for faster inference.

In [ ]:
from examples.train_draft_model import DraftModelTrainingConfiguration, train_draft_model

train_draft_model(
    checkpoint='/content/drive/MyDrive/checkpoints/adapter-final.pt',
    train_data=TRAIN_FILE,
    eval_data=VALID_FILE,
    config=DraftModelTrainingConfiguration(
        epochs=3,
        learning_rate=1e-4
        ),
    checkpoint_dir='/content/drive/MyDrive/checkpoints',
    checkpoint_frequency=1
)

INFO:examples.utils:Loading target (/teacher) model on cuda with precision torch.float32
INFO:examples.utils:Loading draft (/student) model on cuda with precision torch.float32
INFO:examples.utils:Gradient scaling is enabled: False
INFO:examples.utils:Epoch 1/3
Training:   0%|          | 0/15 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.24 GiB. GPU 0 has a total capacity of 79.32 GiB of which 15.18 GiB is free. Process 204572 has 64.13 GiB memory in use. Of the allocated memory 63.29 GiB is allocated by PyTorch, and 343.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Step 6: Evaluate Adapter

Test adapter quality.

In [ ]:
from examples.generate import generate_content, GenerationConfiguration
from examples.messages import Message

output = generate_content(
    [[Message.from_system('You are a helpful assistant generating Übersicht widgets.'),
      Message.from_user('Generate a widget for a toggle switch with customizable colors')]],
    GenerationConfiguration(temperature=0.0, max_new_tokens=128),
    checkpoint='/content/drive/MyDrive/checkpoints/adapter-final.pt',
#    draft_checkpoint='/content/drive/MyDrive/checkpoints/draft-model-final.pt'
)
print(output[0].response)

```javascript
import { html } from 'lit';
import { css } from 'lit/css';

const toggleSwitch = html`
  <div class="toggle-switch">
    <input type="checkbox" class="toggle-switch__input" />
    <div class="toggle-switch__indicator"></div>
  </div>
`;

const toggleSwitchCss = css`
  .toggle-switch {
    display: flex;
    align-items: center;
    justify-content: center;
    margin


## Step 7: Export Adapter

Export to `.fmadapter`.

In [ ]:
# Verify we're using coremltools-tested versions
# (These should already be installed from Cell 4)
print("="*60)
print("Verifying coremltools-tested versions are installed...")
print("="*60)

!pip show torch scikit-learn xgboost tensorflow 2>&1 | grep -E "(Name|Version)" || echo "Some packages not found"

print("\n" + "="*60)
print("If versions above match tested versions, export should have no warnings")
print("Tested versions: torch==2.5.0, scikit-learn<=1.5.1, xgboost==1.4.2, tensorflow==2.12.0")
print("="*60)

In [ ]:
from export.export_fmadapter import Metadata, export_fmadapter

metadata = Metadata(
    author='Your Name',
    description='Adapter for generating Übersicht widgets'
)

export_fmadapter(
    output_dir='/content/drive/MyDrive/AITraining/US_E2E/TrainedAdapter',
    adapter_name=f'adapter_{DATA_SET}',
    metadata=metadata,
    checkpoint='/content/drive/MyDrive/checkpoints/adapter-final.pt',
#    draft_checkpoint='/content/drive/MyDrive/checkpoints/draft-model-final.pt'
)

'/content/drive/MyDrive/AITraining/US_E2E/TrainedAdapter/uebersicht_widgets.fmadapter'

## Notes
- Upload toolkit/widgets to `/content/drive/MyDrive/toolkit/` and `/widgets/`.
- Update `prompt_map` for all 434 widgets.
- Training: ~30–60 min on T4 (free); ~15–30 min on A100 (Pro, $10/mo).
- Memory: `PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0` mitigates MPS errors; T4 has 16GB VRAM.
- If `tamm` fails, contact Apple Developer Support.
- Run `produce_asset_pack` on MacOS; verify `.fmadapter` locally.